In [3]:
import pandas as pd
import script_reglas
import re 
import unicodedata
import nltk
from nltk.stem.snowball import SnowballStemmer
from numpy import reshape, shape, concatenate, nan
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBClassifier
from sklearn.externals import joblib

# Leer archivo con mensajes, categorías y hora de entrada del mensaje

El archivo debe tener la siguiente estructura: 

In [6]:
respuestas_etiquetas=pd.read_csv('../datos/para_entrenamiento.csv')

In [7]:
respuestas_etiquetas.head()

,Unnamed: 0,hora_ultimo,texto,categ_opi
0,0,18,La solicitud de enviar el SMS por Cobrar a 552...,otra
1,1,18,Te llame y no pude localizarte. Tramita en lin...,otra
2,2,18,59508,otra
3,3,18,Mi bebe,nacimiento
4,4,18,Hola si ya cada integrante tenemos fotos con l...,respuesta


Debe tener una columna de hora de entrada que se llamará hora_ultimo, el texto del mensaje ("texto") y la etiqueta asignada ("categ_opi"). En este archivo ya no deben estar los mensajes que sean clasificados por reglas (script_reglas.py).

In [270]:
train_target=respuestas_etiquetas['categ_opi'].values

In [271]:
train_texto=respuestas_etiquetas['texto'].values

# Definir funciones
Se definen las funciones para procesar el texto y reducir palabras hasta su raíz

In [257]:
def procesa_texto(texto):
    # Esta función manda todo a minúsculas, quita la segunda parte de las urls, quita puntuación y espacios finales
    # Posteriormente, quita emojis del texto, quita acentos y ñs,
    #calcula número de palabras y quita frases de apertura iniciales
    
    texto=texto.lower()
    
    
    part=texto.partition('http') 
    part=part[0]+part[1]+' '+ ' '.join(part[2].split(' ')[1:])
    texto=part
    
    part=texto.partition('bit.ly')
    part=part[0]+part[1]+' '+ ' '.join(part[2].split(' ')[1:])
    texto=part
    
    texto=re.sub('^[ \t]+|[ \t]+$', '', texto) 

    texto=re.sub('[^\w\s]','', texto)

    texto=re.sub('^[ \t]+|[ \t]+$', '', texto)

    texto=script_reglas.give_emoji_free_text(texto)

    texto=unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')
    wc=len(str(texto).split(" "))
    texto=re.sub('ola|buena noche|buenos dias|buenos dia|buen dia|buenas noches|buenas tardes|buenas tarde|buen dia|bien dia|buena tardes|buena tarde|saludos|hola','', texto)
    texto=re.sub('\n',' ', texto)
    texto=re.sub('^[ \t]+|[ \t]+$', '', texto)
    return texto , wc

stemmer = SnowballStemmer("spanish")


def tokenize_and_stem(text):
    #Esta función separa el mensaje por palabras y reduce las palabras a su raíz
    #
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

Aplicamos las funciones y dejamos en un array los features para el modelo

In [258]:
train_texto_stem=[]

In [259]:
wc=[]

for i in range(0, shape(train_texto)[0]):
    wc.append(procesa_texto(train_texto[i])[1])
    train_texto[i]=procesa_texto(train_texto[i])[0]
    train_texto_stem.append(tokenize_and_stem(train_texto[i]))
    train_texto_stem[i]=' '.join(train_texto_stem[i])
wc=reshape(wc, (-1, 1))

hora=respuestas_etiquetas.hora_ultimo.values
hora=reshape(hora, (-1, 1))

Definimos stopwords que se van a quitar del cálculo de features del modelo. los pasamos a minúsculas y quitamos acentos

In [260]:
stop=nltk.corpus.stopwords.words("spanish")

for i in range(0, shape(stop)[0]):
    stop[i]=unicodedata.normalize('NFD', stop[i]).encode('ascii', 'ignore').decode('utf-8')

# Feature engineering

Convertimos el texto a una matriz de TFIDF. Cada columna representa un "token" del vocabulario completo (un token representa cada palabra y combinación de dos palabras). Si la celda es 0, es que no está esa palaba en el mensaje. Si es mayor a 0 sí está. Las palabras pesan menos cuanto más comunes sean en el vocabulario

In [262]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0.006, norm='l2', encoding='utf-8', ngram_range=([1, 2]),
                        stop_words=stop)
tfidf=tfidf.fit(train_texto_stem)

features_stem = tfidf.transform(train_texto_stem)
labels = respuestas_etiquetas.categ_opi
features_stem.shape

(931, 466)

Hay 931 mensajes con 466 palabras 

La matriz TFIDF tiene muchas columnas. Un modelo con tan pocas observaciones no puede ser entrenado con tantos features. Si reducimos las dimennsiones ganamos más poder predictivo al eliminar el ruido que puede sobreajustar el modelo. 

Para reducir las dimensiones usamos una técnica de "latent semantic analysis" que detecta temas subyacentes en los mensajes. Tras probar diferente número de factores, se definió que en 33 factores, la predicción no mejoraba sustancialmente por un factor adicional

In [263]:
features_stem=features_stem.toarray()
pca=TruncatedSVD(n_components=33)
pca=pca.fit(features_stem, features_stem)
features_stem_pca=pca.transform(features_stem)

Se agrega a la matriz de dimensiones, los features de número de palabras y hora de entrada

In [264]:
x_train=concatenate((features_stem_pca, wc), axis=1)

In [265]:
x_train=concatenate((x_train, hora), axis=1)

# Definición de clasificador

El clasificador que mejor funciona en este caso es el XGBoost. Éste es un ensamble de árboles que utiliza Gradient Descent para minimizar el error. Se cambia la métrica de error a Area Under Curve, para evitar que todas las predicciones se vayan a las clases más numerosas. Posteriormente, se entrena el modelo aplicando un .fit

In [266]:
clasificador=XGBClassifier(metrics='auc')
clasificador=clasificador.fit(X=x_train,y=train_target)


In [267]:
clasificador.classes_

array(['emergencia', 'informacion', 'nacimiento', 'otra', 'otra_queja',
       'pregunta', 'pregunta_busca trabajo', 'pregunta_medica',
       'respuesta'], dtype=object)

# Se exportan los PKL de matriz TFIDF, reducción de dimensiones y clasificador entrenado

In [117]:
joblib.dump(tfidf, './modelo/mat_tfidf.pkl') #1
joblib.dump(pca, './modelo/pca.pkl')  #2
joblib.dump(clasificador, './modelo/modelo.pkl') # 3
 

['./modelo/modelo.pkl']